In [33]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from clust import *
import umap
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import umap
import hdbscan
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
from hyperopt import fmin, tpe, space_eval, Trials, STATUS_OK, partial
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials


# Analysis

## Load the corpus

In [34]:
corpus = pd.read_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_disto.csv")
corpus["thought"]
corpus_neg = list(corpus["thought"])
corpus


,Unnamed: 0,thought,original_label,cleaned_thought
0,0,Someone I trusted stole something valuable of ...,emotional reasoning,someone trusted stole something valuable mine ...
1,1,She doesn't respect me.,overgeneralizing,doesnt respect
2,2,My friend is ignoring his recently-deceased wife.,disqualifying the positive,friend ignoring recentlydeceased wife
3,3,he took me for granted!,"labeling,catastrophizing",took granted
4,4,We will not be able to complete the work that ...,catastrophizing,able complete work need done
...,...,...,...,...
917,917,Not having made rent and having to borrow mone...,Labeling,made rent borrow money already debt mother fai...
918,918,I’m a loser I hate myself and I’m ready to die,Labeling,im loser hate im ready die
919,919,People around me do not need my presence I'm w...,Emotional reasoning,people around need presence im worthless
920,920,I realize that I still don’t have anyone who I...,Overgeneralizing,realize still dont anyone could spend time wit...


## Sentence Embedding
https://www.sbert.net/index.html

In [35]:
"""
Sentences are mapped to sentence embeddings with three different pre-trained model
https://huggingface.co/sentence-transformers/all-mpnet-base-v2
https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2
https://huggingface-co.translate.goog/sentence-transformers/all-distilroberta-v1?_x_tr_sl=en&_x_tr_tl=fr&_x_tr_hl=fr&_x_tr_pto=sc

"""
i=0

# specify the model
model = 'all-mpnet-base-v2'
model_2 = 'all-MiniLM-L12-v2'
model_3 = 'All-Distilroberta-v1'
models = [model,model_2,model_3]

# Load the model
embedder = SentenceTransformer(model)
embedder_2 = SentenceTransformer(model_2)
embedder_3 = SentenceTransformer(model_3)

# encode
corpus_embeddings = embedder.encode(corpus_neg)
corpus_embeddings_2 = embedder_2.encode(corpus_neg)
corpus_embeddings_3 = embedder_3.encode(corpus_neg)

# merge 3 embeding
embeddings = [corpus_embeddings, corpus_embeddings_2, corpus_embeddings_3]

for embedding in embeddings:
    print(f"Model: {models[i]}")
    print(f"Shape: {embedding.shape}")
    i=i+1


Model: all-mpnet-base-v2
Shape: (922, 768)
Model: all-MiniLM-L12-v2
Shape: (922, 384)
Model: All-Distilroberta-v1
Shape: (922, 768)


## Export embedding

In [25]:
index = 0
for embedding in embeddings:
    umap_data = umap.UMAP(n_components=2,random_state=42).fit_transform(embedding)
    x, y = umap_data.T
    corpus[f"{models[index]} x"] = x
    corpus[f"{models[index]} y"] = y
    index = index +1
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_embedding.csv")



/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


## Clustering


### K-mean clustering


In [241]:
# Parameters
n_clusters = 18 # number of cluster
dimension_reduction = True # reduce the dimension before clustering
dim = 3 # number of dimension to reduce embedding to.
results_k_mean = 0

In [243]:
index = 0
cluster_dict_k = {}

for embedder in embeddings:
    kmeans = 0
    print(f"run model {models[index]}")   
    corpus_embeddings_reduce = embedder

    # Dimension reduction
    if dimension_reduction:
        print(f"dimension before reduction {corpus_embeddings_reduce.shape}")
        # Reduce the embedding with UMAP
        model = UMAP(n_components=dim,random_state=42)
        corpus_embeddings_reduce = model.fit_transform(corpus_embeddings_reduce)
        print(f"dimension after reduction {corpus_embeddings_reduce.shape}")

    # Cluster with KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state = 42)
    kmeans.fit(corpus_embeddings_reduce)
    corpus[f"{models[index]} k-mean"] = kmeans.labels_
    print(kmeans.labels_)
    # Get cluster labels
    cluster_labels = cluster.labels_

    
    # Reduce the dimension with UMAP for visualization
    model2 = UMAP(n_components=2,random_state=42)
    embeddings_2d = model2.fit_transform(corpus_embeddings_reduce)
    x, y = embeddings_2d.T  
    corpus[f"{models[index]} x"] = x
    corpus[f"{models[index]} y"] = y




    index = index + 1


run model all-mpnet-base-v2
dimension before reduction (922, 768)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



dimension after reduction (922, 3)
[ 3 13  1  1 15  5 16  5  1  2  8 14  3  2 11  6  6 15  4  2 17  5  7 13
 11  8 11 11  2 10  4  4  9  1 15 14 11  0 15  8 16  4 13 13  1  8  0 13
 13 13  1 11 11  2 14 17  1 15  8  2 17  0 16 15 10 11 15 14  5  7 15 15
 15  2 14 13  0  1  0 16  7 10 16 13 13  0 15 13 17  6  5 15  1 14  7 11
 17 10 15  5  2  6 15  0  2  8 15 17  0  6  7  0  0  0  5  8 16 10  4 10
 13  0 17 10  3  8  0 11 15  4 11 15  2  5 16  5 11 15  6 10  3  7  7  5
  7 12 15 16 16 17 16  1  1  1  0 14  5  1 17 17 12  6 14 15 14 11  2  1
 15  7 15  5  5 15  6  8  4  6 12 11  3 16 17 10  7  0 16 15 15 11  1  1
  8  1  1  1  4 15 13 13  3 14 15  6 10  1  1 11 11  8 13 13 13  4 13  2
 13  6  6 15 17 12  1 15  1  2  5 13 16 11 17  0 13  2 10  3  2  3 14  1
  1 14  3 13 13  4 13  7  4 15  7 15 15 16 15 16 10 11  6  1  4 10 17  5
 15  7 10 16  3 17  0  2  4  6 15 11 14  9  4  2 15 10 15  5 16 13  7  7
  7  7  7  0  0  0 11 11 13 14  8 15  8  8  8 17  8  4  4 11 12 15  7  7
  6  7  0 15 15 

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



run model all-MiniLM-L12-v2
dimension before reduction (922, 384)


...

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



run model All-Distilroberta-v1
dimension before reduction (922, 768)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



dimension after reduction (922, 3)
[11  8  3 17  6  5 12  1  3 14  4 15 11  4 14  0 14  4 16 14 16  1 12  8
 16 12  4 14  4  3 16 16 10 17  2  9  2 13  4 12  5 16  3  8 17 12  6  8
  8  8  8  2  2 14 15  7 17  4 12  2  7 11 12  6 16  2  1  9  1  6  4  4
  4 14  9 15 11 17  6 11  6  0 11  8  3  6  6  8  7  6  1 12  3  9  6  2
  7  2  4  1 14 14  4  2 14 12  4  1  6  7  6  6 13  6  1 12 11  3 16 16
  3  2  7  3 15  1 13 16  4 16 16  2  4  1  5  1  7  4 14 11 15 17  1  1
  6 13  4  5  5  6 11  3 17  3  2  9  1  3  7  7 13 14  1  6  9  0 14  3
  4  6 10  1  1  4 14 12 16  0 13 16 15  5  7  7  1  9 12  4  4 14 17  3
  4 17 17 17 16  4  8  8 15  0  4 14  0 17 11  2  2  7  8  8  8 16  8  0
  8  0  0  4  7  4 17 10 17 14  1  8  5  4 11  6  8  2  2 15 14 15  9  3
  3  9 15  8  8 16  8  6 16  6  6  1  4 11  4 11  2  9 14  3  2  5 16  1
  4  6  2 12  8 16  6 16 16 14  4  1 15 10 16 14  4  0 14  1 11  8  6  6
  6  6  6  6  6  6  2  2 15  9  4  4 12 12  4  6 12 16 16 14  7  4  6 12
 14  5 11  1 14 

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [244]:
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_kmean.csv")


### Aglomerative Clustering

In [14]:
# Parameters
dimension_reduction = True # reduce the dimension before clustering
dim = 3 # number of dimension to reduce embedding to.

In [15]:
index = 0
cluster_dict_k = {}

for embedder in embeddings:
    kmeans = 0
    print(f"run model {models[index]}")   
    corpus_embeddings_reduce = embedder

    # Dimension reduction
    if dimension_reduction:
        print(f"dimension before reduction {corpus_embeddings_reduce.shape}")
        # Reduce the embedding with UMAP
        model = UMAP(n_components=dim,random_state=42)
        corpus_embeddings_reduce = model.fit_transform(corpus_embeddings_reduce)
        print(f"dimension after reduction {corpus_embeddings_reduce.shape}")

    corpus_embeddings_reduce = corpus_embeddings_reduce /  np.linalg.norm(corpus_embeddings_reduce, axis=1, keepdims=True)

    # Cluster with KMeans
    clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5)
    clustering_model.fit(corpus_embeddings_reduce)
    corpus[f"{models[index]} aglo"] = clustering_model.labels_
    print(clustering_model.labels_)
    
    # Reduce the dimension with UMAP for visualization
    model2 = UMAP(n_components=2,random_state=42)
    embeddings_2d = model2.fit_transform(corpus_embeddings_reduce)
    x, y = embeddings_2d.T  
    corpus[f"{models[index]} x"] = x
    corpus[f"{models[index]} y"] = y




    index = index + 1


run model all-mpnet-base-v2
dimension before reduction (922, 768)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


dimension after reduction (922, 3)
[1 0 0 0 1 1 1 1 0 2 2 1 1 2 2 2 2 2 0 2 0 2 2 0 2 2 2 2 2 0 0 0 2 0 2 1 2
 1 2 1 1 0 0 0 0 2 1 0 0 0 0 2 2 2 1 0 0 2 1 2 0 1 1 1 0 2 2 1 1 1 2 2 2 2
 1 0 1 0 1 1 1 0 1 0 0 1 2 0 0 2 1 2 0 1 1 2 0 0 2 1 2 2 2 1 2 2 2 2 1 2 1
 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 2 0 0 2 2 1 1 1 0 2 1 0 0 2 1 1 1 1 2 1
 1 1 1 0 0 0 1 1 1 0 0 2 1 2 1 2 1 0 2 0 2 1 2 1 1 2 2 2 0 1 1 0 0 1 0 0 1
 2 1 1 2 2 0 0 2 0 0 0 0 2 0 0 0 1 2 2 0 0 0 2 2 2 0 0 0 0 0 2 0 2 2 2 0 1
 0 2 0 2 1 0 1 2 1 1 0 2 0 0 2 1 1 0 0 1 1 0 0 0 0 2 0 2 2 2 2 1 2 1 2 2 2
 0 0 2 0 1 2 1 2 1 0 0 1 2 0 2 2 2 1 2 0 2 2 2 2 1 1 0 1 1 1 1 1 1 1 1 2 2
 0 1 2 2 2 2 2 0 1 0 0 0 1 2 2 2 2 2 1 2 2 2 2 1 2 2 0 2 1 1 1 1 1 1 1 1 2
 1 1 0 1 1 2 2 2 2 1 1 1 0 0 0 2 0 1 1 1 1 1 1 1 2 0 1 1 1 1 1 1 1 1 0 1 1
 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 2 1 1 2 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1
 1 1 2 2 1 0 1 2 1 1 2 1 1 1 1 2 2 1 2 0 2 0 0 1 1 0 2 2 2 2 2 2 2 1 0 1 0
 0 2 2 1 1 0 2 0 1 2 2 2 2 2 0 2 2 2 0 0 0 1 1 0 1 0 0 1 0 2 0 0 

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


run model all-MiniLM-L12-v2
dimension before reduction (922, 384)


...

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


run model All-Distilroberta-v1
dimension before reduction (922, 768)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


dimension after reduction (922, 3)
[3 1 1 1 0 0 0 0 1 2 4 1 3 4 2 2 2 0 2 2 3 0 0 1 2 0 0 2 4 1 3 2 4 1 0 2 0
 3 4 0 0 2 1 1 1 0 0 1 1 1 1 0 0 2 1 3 1 0 0 2 3 3 0 0 2 0 0 2 0 0 0 4 4 2
 2 1 3 1 0 3 0 2 3 1 1 0 0 1 3 0 0 0 1 2 0 3 3 3 4 0 2 2 4 3 2 0 4 0 0 0 0
 0 3 0 0 0 3 1 1 2 1 3 3 1 1 0 3 2 4 3 3 2 2 0 0 0 3 4 2 3 1 1 0 0 0 3 4 0
 0 0 3 1 1 1 2 2 0 1 3 3 3 2 0 0 2 2 2 1 4 0 4 0 0 0 2 0 2 2 3 2 1 0 3 3 0
 2 0 0 4 2 1 1 4 1 1 1 2 4 1 1 1 2 4 2 2 1 3 2 2 3 1 1 1 3 1 2 1 2 2 4 3 0
 1 4 1 2 0 1 0 0 3 0 1 2 2 1 2 1 2 1 1 2 1 1 1 2 1 0 2 0 0 0 4 3 4 3 3 2 2
 1 2 0 3 0 4 0 2 0 1 3 0 2 3 2 0 0 1 4 2 2 4 2 2 0 3 1 0 0 0 0 0 0 0 0 3 2
 1 2 0 0 0 0 4 0 0 3 3 2 3 4 0 0 4 0 3 4 2 2 4 0 4 2 2 4 3 3 4 2 0 0 0 0 0
 3 3 1 3 3 0 0 0 0 0 3 3 3 3 3 2 3 3 3 0 0 0 0 0 2 3 0 0 3 3 3 3 0 3 3 3 3
 3 1 1 0 2 2 1 1 1 1 3 3 0 0 3 1 3 2 3 3 3 3 3 1 1 1 1 1 2 2 2 1 1 3 3 3 3
 3 3 0 0 3 3 0 0 0 2 4 3 0 0 0 2 4 1 2 1 2 3 2 0 3 1 2 0 2 0 4 2 2 3 1 0 1
 1 0 4 0 1 1 2 2 4 0 4 0 0 0 1 2 0 0 1 1 1 0 3 2 2 1 3 2 1 0 1 3 

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [16]:
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_aglom.csv")


### DBSCAN

### Random Estimation

### try differents parameter and select for each model the most appropriate

In [4]:
cluster_dict = {}
random_dict = {}
index = 0
space = {
        "n_neighbors": range(12,16),
        "n_components": range(3,7),
        "min_cluster_size": range(2,16),
        "random_state": 42,
        "min_sample": None
    }

In [5]:
for embedding in embeddings:
    embedding_name = f"embedding_{index}"
    random_dict[embedding_name] = random_search(embedding,space, 100)
    index=index+1


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning

In [12]:
random_dict["embedding_2"].sort_values(by='cost').head(50)


,run_id,n_neighbors,n_components,min_cluster_size,label_count,cost
39,39,14,3,12,4,0.069414
8,8,14,3,14,4,0.087852
15,15,14,3,14,4,0.087852
94,94,15,4,14,3,0.091106
42,42,14,3,13,4,0.105206
32,32,14,3,13,4,0.105206
98,98,15,5,2,129,0.224512
31,31,15,5,2,129,0.224512
12,12,12,5,2,122,0.236443
21,21,12,3,2,135,0.236443


In [17]:
# Select the parameter given the result above
n_neighbors = [15,13,14]
n_components = [5,4,3]
min_cluster_size = [8,5,13]
index = 0
cluster_dict_k = {}

for embedder in embeddings:
    clusters_manual = generate_clusters(embedder,n_neighbors = n_neighbors[index], 
                                                n_components = n_components[index], 
                                                min_cluster_size = min_cluster_size[index], 
                                                random_state=42)
    labels_manual, cost_manual = score_clusters(clusters_manual)
    print(labels_manual)
    print(cost_manual)
    corpus[f"{models[index]} hdbscan_manual"] = clusters_manual.labels_
    cluster_dict["label_manual1"] =  clusters_manual

    umap_data = umap.UMAP(n_neighbors=15, 
                            n_components=2, 
                            min_dist = 0.1,  
                            metric='cosine',
                            random_state=42).fit_transform(corpus_embeddings)
    x, y = umap_data.T
    corpus[f"{models[index]} x"] = x
    corpus[f"{models[index]} y"] = y
    index = index + 1


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


17
0.2885032537960954


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


38
0.2809110629067245


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


4
0.1052060737527115


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [18]:
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_hdbscan.csv")


### Bayesian Estimation

In [41]:
best_params_dict = {}
best_clusters_dict = {}
trials_dict = {}

hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(2,20)),
    "n_components": hp.choice('n_components', range(2,20)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,8)),
    "random_state": 42
}

label_lower = 5
label_upper = 60
max_evals = 200

In [42]:
    index=0
    for embedding in embeddings:
        embedding_name = f"embedding_{index}"
        best_params_use, best_clusters_use, trials_use = bayesian_search(embedding, 
                                                                    space=hspace, 
                                                                    label_lower=label_lower, 
                                                                    label_upper=label_upper, 
                                                      max_evals=max_evals)
        best_params_dict[embedding_name] = best_params_use
        best_clusters_dict[embedding_name] = best_clusters_use
        trials_dict[embedding_name] = trials_use
        cluster_dict[f"label_use{index}"] =    best_clusters_dict[embedding_name]
        corpus[f"{models[index]} hdbscan_bayesian_optimisation"] = best_clusters_use.labels_

        umap_data = umap.UMAP(n_neighbors=15, 
                          n_components=2, 
                          min_dist = 0.1,  
                          metric='cosine',
                          random_state=42).fit_transform(embedding)
        x, y = umap_data.T
        corpus[f"{models[index]} x"] = x
        corpus[f"{models[index]} y"] = y
        index=index+1


  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")



  0%|          | 1/200 [00:01<04:33,  1.37s/trial, best loss: 0.30726681127982647]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")



  1%|          | 2/200 [00:02<04:50,  1.47s/trial, best loss: 0.30726681127982647]

...

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")



100%|█████████▉| 199/200 [04:46<00:01,  1.46s/trial, best loss: 0.15]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")



100%|██████████| 200/200 [04:47<00:00,  1.44s/trial, best loss: 0.15]
best:
{'min_cluster_size': 7, 'n_components': 10, 'n_neighbors': 5, 'random_state': 42}
label count: 3


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [43]:
corpus.to_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/corpus_hdbscan_bayesian_optimisation.csv")
